## Chunking :D
## Using more data
(Ozone just happens to come in huge files that make it easy to make a point here)

In [1]:
import numpy as np
import xarray as xr
import time

In [2]:
from dask.distributed import Client
client = Client(threads_per_worker=1,memory_limit=0)
client.amm.start()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 7,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36985,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:43369,Total threads: 1
Dashboard: /proxy/33947/status,Memory: 0 B
Nanny: tcp://127.0.0.1:36073,


In [ ]:
import intake
catalog = intake.cat.access_nri

catalog['era5_rt52'].search(variable='o3',
                            product='era5-1-monthly-averaged',
                           ).to_dask(xarray_open_kwargs={'chunks':{}}).o3

In [ ]:
ozone = xr.open_dataset('/g/data/rt52/era5/pressure-levels/reanalysis/o3/2000/o3_era5_oper_pl_20000101-20000131.nc',
                        # chunks = None, # the default
                        chunks={}, # the default for open_mfdataset
                        # chunks={'longitude':-1,'time':12}, # should help
                        # chunks={'longitude':-1,'time':-1}, # dumb
                       )



In [ ]:
%time ozone.o3.sel(level=1,latitude=slice(-45,-90)).mean('longitude').load()

# Exercises
recommend using "Large" ARE session -- 7 cores

## Can you pick good chunk sizes to speed up the calculation of global mean surface temperature?

#### 1. ACCESS-CM2 pre-industrial control run (monthly)
```python
xr.open_mfdataset('/g/data/fs38/publications/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/piControl/r1i1p1f1/Amon/tas/gn/latest/*.nc', parallel=True, chunks={}).tas
# OR
catalog['cmip6_fs38'].search(source_id='ACCESS-ESM1-5',
                             experiment_id='piControl',
                             variable_id='tas',
                             version='latest',
                             frequency='mon'
                                   ).to_dask(xarray_open_kwargs={'chunks':{}}).tas
```

#### 2. ACCESS-CM2 pre-industrial control run (daily)
```python
xr.open_mfdataset('/g/data/fs38/publications/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/piControl/r1i1p1f1/day/tas/gn/latest/*.nc', parallel=True, chunks={}).tas
# OR
catalog['cmip6_fs38'].search(source_id='ACCESS-ESM1-5',
                             experiment_id='piControl',
                             variable_id='tas',
                             version='latest',
                             frequency='day'
                             ).to_dask(xarray_open_kwargs={'chunks':{}}).tas

```

#### 3. ERA5 hourly
```python
xr.open_mfdataset('/g/data/rt52/era5/single-levels/reanalysis/2t/*/*.nc').t2m
# OR
catalog['era5_rt52'].search(variable='2t',
                            product='era5-monthly-averaged'
                           ).to_dask(xarray_open_kwargs={'chunks':{}}).t2m
```